In [1]:
import os
import h5py
import numpy as np
from typing import List, Dict
from stim_pyper.processing_utils.optimizer_postprocessing import process_vta
from scipy.io import loadmat
from glob import glob

Helper Functions

Assumes a leaddbs BIDS directory, with patients stored in the same lead group. May need to modify the get_stimulations function slightly if your dataset is organized differently.

In [2]:
import os
from glob import glob
from scipy.io import loadmat
import h5py
import numpy as np
def _h5_to_dict(group):
    """Recursively turn an h5 group into a nested dict of numpy arrays."""
    out = {}
    for k, v in group.items():
        if isinstance(v, h5py.Dataset):
            out[k] = v[()]           # load actual data
        else:
            out[k] = _h5_to_dict(v)  # recurse into subgroup
    return out

def get_mat_file(base_path, ftype='stim'):
    """
    Quick & dirty:
    - Try scipy.io.loadmat
    - If it's a v7.3 file (NotImplementedError), use h5py instead
    """
    json_data = []

    for mat_file_path in glob(base_path):
        try:
            data = loadmat(mat_file_path, simplify_cells=True)
        except:
            continue
            # with h5py.File(mat_file_path, "r") as f:
            #     data = _h5_to_dict(f)
        json_data.append({
            'dir_name': os.path.dirname(mat_file_path),
            'data': data if ftype == 'stimulation' else data['reco']['mni']['coords_mm'][:]
        })

    return json_data

In [3]:
def get_v(stim):
    try:
        S = stim['S']
        right_elec = S['Rs1']
        right_amp = np.array(right_elec['amp'])
        
        right_v = []
        for key, value in sorted(right_elec.items(), key=lambda x: (x[0][0], int(x[0][1:])) if x[0][1:].isdigit() else (x[0][0], x[0][1:])):
            if key.startswith('k'):
                perc = np.array(value.get('perc', np.nan))
                try:
                    right_v.append(right_amp * perc / 100)
                except:
                    right_v.append(0)

        left_elec = S['Ls1']
        left_amp = np.array(left_elec['amp'])
        left_v = []
        for key, value in sorted(left_elec.items(), key=lambda x: (x[0][0], int(x[0][1:])) if x[0][1:].isdigit() else (x[0][0], x[0][1:])):
            if key.startswith('k'):
                perc = np.array(value.get('perc', np.nan))
                try:
                    left_v.append(left_amp * perc / 100)
                except:
                    left_v.append(0)
        
        v = {'right': right_v, 'left': left_v}
    except:
        with h5py.File(stim, 'r') as f:
            S = f['S']
            right_elec = S['Rs1']
            right_amp = np.array(right_elec['amp'])
            right_v = []
            for key, value in sorted(right_elec.items(), key=lambda x: (x[0][0], int(x[0][1:])) if x[0][1:].isdigit() else (x[0][0], x[0][1:])):
                if key.startswith('k'):
                    perc = np.array(value.get('perc', np.nan))
                    try:
                        right_v.append(right_amp * perc / 100)
                    except:
                        right_v.append(0)

            left_elec = S['Ls1']
            left_amp = np.array(left_elec['amp'])
            left_v = []
            for key, value in sorted(left_elec.items(), key=lambda x: (x[0][0], int(x[0][1:])) if x[0][1:].isdigit() else (x[0][0], x[0][1:])):
                if key.startswith('k'):
                    perc = np.array(value.get('perc', np.nan))
                    try:
                        left_v.append(left_amp * perc / 100)
                    except:
                        left_v.append(0)
            
            v = {'right': right_v, 'left': left_v}
    return v

In [4]:
def leaddbs_to_stimpyper(stimulations, reconstructions, output_dir, val=None):
    """Processes the data and calls handle_nii_map for each combination of lambda and weight."""
    for reco, stim in zip(reconstructions, stimulations):
        coords_list = ['coords1', 'coords2']
        side = ['right', 'left']
        stimdata = get_v(stim['data'])
        for i, coords in enumerate(coords_list):
            optima_ampers = np.copy(stimdata[side[i]])
            electrode_coords = np.array(reco['data'][i])
            electrode_idx = i
            out_dir = os.path.join(output_dir, reco['dir_name'], 'stimulations/MNI_ICBM_2009b_NLIN_ASYM/stimpyper/gs_20180403170745' f'{side[i]}')
            process_vta(optima_ampers, electrode_coords, electrode_idx, out_dir)

Start Here

In [5]:
out_dir = '/Volumes/HowExp/datasets/03h_ADVANCE_Alzheimer_DBS/test'
stimulations = get_mat_file(base_path="/Volumes/HowExp/datasets/03h_ADVANCE_Alzheimer_DBS/subjects/sub-*/stimulations/MNI_ICBM_2009b_NLIN_ASYM/gs_20180403170745/stimparameters.mat", ftype='stimulation')
reconstructions = get_mat_file(base_path="/Volumes/HowExp/datasets/03h_ADVANCE_Alzheimer_DBS/subjects/sub-*/ea_reconstruction.mat", ftype='reconstruction')
leaddbs_to_stimpyper(stimulations, reconstructions, output_dir=out_dir)